In [1]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /home/parker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/parker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import json
import string
import random
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import torch
from torch import nn
from textblob import Word
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

/home/parker/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
with open("AllPrintings.json", encoding='utf-8') as f:
    data = json.load(f)

In [4]:
cards = {}
for group_key in list(data["data"].keys()):
    for card in data["data"][group_key]["cards"]:
        if "foreignData" in list(card.keys()):
            del card["foreignData"]
        name = card["name"]
        cards[name] = card

In [5]:
def two_or_fewer_colors(text):
    shared_characters = ''.join(set("RBGUW").intersection(text)) 
    return len(shared_characters) in [1,2]

def filter_card(key):    
    return ("text" in list(cards[key].keys()) 
            and "//" not in key 
            and cards[key]["rarity"] not in ["bonus", "special"]
            and any([t in cards[key]["type"] for t in ["Planeswalker", "Enchantment", "Sorcery", "Instant", "Artifact", "Creature"]])
            and "Land" not in cards[key]["type"]
            and "manaCost" in list(cards[key].keys())
            and two_or_fewer_colors(cards[key]["colors"])
            #below names have no mana_cost, strangely
            and key not in ["Pact of Negation", "Summoner's Pact", "Rograkh, Son of Rohgahh", "Intervention Pact", "Slaughter Pact", "Pact of the Titan"])
    
cards = {key : cards[key]
         for key in list(cards.keys())
         if filter_card(key)}
print(len(list(cards.keys())))
card_texts = {key: cards[key]["text"] for key in list(cards.keys())}

17742


In [6]:
def process_color(text):
    colors = []
    for c in text.split("{"):
        c = c.replace("}", "")
        if c.isdigit():
            n_blank = int(c)
            colors.append(n_blank*"M")
        else:
            colors.append(c)
    colors = [c for c in colors if c != '']
    colors.sort()
    s = "".join(colors)
    return f"{s}"

def process_before_colon(text):
    length_check = len(text.split(" ")) >= 6
    if length_check:
        text = text.replace(",", "")
    precolon_elements = ["<precolon>"]
    for word in text.split(" "):
        if "{" in word:
            precolon_elements.append("<color>")
            precolon_elements.append(process_color(word))
            precolon_elements.append("</color>")
        elif "[" in word:
            word = word.replace("[", "")
            word = word.replace("]", "")
            precolon_elements.append("<symbol>")
            precolon_elements.append(word)
            precolon_elements.append("</symbol>")
        else:
            precolon_elements.append(add_tags(word))
    precolon_elements.append("</precolon>")
    return " ".join(precolon_elements)
            

def add_tags(text):
    
    sections = text.split("(")
    if len(sections) > 2 or (len(sections) == 2 
                             and sections[1].index(")") not in [len(sections[1])-1,len(sections[1])-2]):
        raise ValueError(text)
    elif len(sections) == 2:
        second_section = sections[1].replace(")", "")
        return f"{add_tags(sections[0])} <reminder> {add_tags(second_section)} </reminder>"
    
    if ":" in text:
        sections = text.split(":")
        return f"{process_before_colon(sections[0])} {add_tags(sections[1])}"
    
    tags = []
    if "." in text:
        tags.append("<sentence>")
        
    color_or_tag = False
    for word in text.split(" "):
        if word in ["<color>", "<symbol>", "</color>", "</symbol>"]:
            color_or_tag = not color_or_tag
        if "." in word:
            word = word.replace(".", "")
        if "<" not in word and not color_or_tag:
            word = word.lower()
        tags.append(word)
    if "." in text:
        tags.append("</sentence>")
    
    return " ".join(tags)

In [7]:
stop = stopwords.words('english')
punctuation = [x for x in string.punctuation if x not in "+/,<>_{}:.'\""]
lemmatized_card_texts = {}
for index, key in tqdm(enumerate(list(card_texts.keys()))):
    t = card_texts[key]
    t = t.replace(key, "<CARD_NAME>")
    t = t.replace('"', "\n")
    t = t.strip()
    t = "".join([char for char in list(t) if char not in punctuation])
    print(key)
    t = [add_tags(line) for line in t.split("\n")]
    tags = ["<card>"]
    for index, line in enumerate(t):
        line_tag = "line" if "•" not in line else "bullet"
        tags.append(f"<{line_tag}>")
        tags.append(line.replace("•", ""))
        tags.append(f"</{line_tag}>")
    tags.append("</card>")
    t = " ".join(tags)
    t = t.replace("<card_name>", "<CARD_NAME>")
    t = t.replace("<CARD_NAME>s", "<CARD_NAME>")
    t.replace(",", "")
    t = str(Word(t).lemmatize())
    lemmatized_card_texts[str(key)] = t
    
    t = "".join([char.lower() for char in list(t) if char not in punctuation])
    t = t.replace('[^\w\s]','')
    #t = " ".join([word.lower() for word in t.split(" ") if word not in stop])
#     t = " ".join([word.lower() for word in t.split(" ")])
#     t = str(Word(t).lemmatize())
#     lemmatized_card_texts.append(t.split(" "))

# model = Word2Vec(lemmatized_card_texts, min_count=1,size=75,workers=4)
# model.save("word2vec.model")
# model.save("model.bin")

e Sliver
Saffi Eriksdotter
Academy Rector
Archery Training
Capashen Standard
Capashen Templar
Fend Off
Field Surgeon
Flicker
Jasmine Seer
Mask of Law and Grace
Opalescence
Reliquary Monk
Replenish
Scent of Jasmine
Tethered Griffin
Tormented Angel
Voice of Duty
Voice of Reason
Wall of Glare
Aura Thief
Brine Seer
Bubbling Beebles
Disappear
Donate
Fatigue
Fledgling Osprey
Illuminated Wings
Iridescent Drake
Kingfisher
Mental Discipline
Metathran Elite
Metathran Soldier
Private Research
Rayne, Academy Chancellor
Scent of Brine
Treachery
Bubbling Muck
Carnival of Souls
Chime of Night
Disease Carriers
Dying Wail
Encroach
Festering Wound
Lurking Jackals
Nightshade Seer
Phyrexian Monitor
Plague Dogs
Rapid Decay
Scent of Nightshade
Slinking Skirge
Squirming Mass
Twisted Experiment
Yawgmoth's Bargain
Aether Sting
Cinder Seer
Colos Yearling
Covetous Dragon
Flame Jet
Goblin Berserker
Goblin Festival
Goblin Marshal
Goblin Masons
Incendiary
Landslide
Mark of Fury
Repercussion
Wake of Destruction
Wild

In [8]:
lemmatized_card_texts

on> <sentence>  <CARD_NAME> gains shroud until end of turn and doesn't untap during your next untap step tap <CARD_NAME> a creature with shroud can't be the target of spells or abilities </sentence> </line> </card>",
 'Hydroblast': "<card> <line> choose one — </line> <bullet> <sentence>  counter target spell if it's red </sentence> </bullet> <bullet> <sentence>  destroy target permanent if it's red </sentence> </bullet> </card>",
 'Juxtapose': '<card> <line> <sentence> you and target player exchange control of the creature you each control with the highest mana value then exchange control of artifacts the same way if two or more permanents a player controls are tied for highest, their controller chooses one of them </sentence> </line> </card>',
 'Krovikan Sorcerer': '<card> <line> <precolon> <color> T, </color> discard a nonblack card </precolon> <sentence>  draw a card </sentence> </line> <line> <precolon> <color> T, </color> discard a black card </precolon> <sentence>  draw two cards

In [9]:
names = []
rarities = []
colors = []
card_types = []
mana_costs = []
powers = []
toughnesses = []
loyalties = []
texts = []

for name, feature_dict in cards.items():
    names.append(name)
    rarities.append(feature_dict["rarity"])
    colors.append("".join(feature_dict["colors"]))
    mana_costs.append(process_color(feature_dict["manaCost"]))
    
    card_type = feature_dict["type"]
    for t in ["Planeswalker", "Enchantment", "Sorcery", "Instant", "Artifact", "Creature"]:
        if t in card_type:
            card_type = t
    card_types.append(card_type)
    
    toughness = None if card_type != "Creature" else feature_dict["toughness"]
    toughnesses.append(toughness)
    power = None if card_type != "Creature" else feature_dict["power"]
    powers.append(power)
    loyalty = None if card_type != "Planeswalker" else feature_dict["loyalty"]
    loyalties.append(loyalty)
    
    texts.append(lemmatized_card_texts[name])
    
cards_df = pd.DataFrame({"name":names, "rarity":rarities, "color":colors, "type":card_types,
                         "mana_cost":mana_costs, "power":powers, "toughness":toughnesses, 
                         "loyalty":loyalties, "text":texts})

cards_df.to_csv("cards.csv", index=False)

In [10]:
cards_df['text'][:10].values

array(['<card> <line> <sentence> first strike this creature deals combat damage before creatures without first strike </sentence> </line> <line> <sentence> when <CARD_NAME> enters the battlefield, you gain 1 life for each card in your graveyard </sentence> </line> </card>',
       '<card> <line> flying </line> <line> <sentence> when <CARD_NAME> enters the battlefield, you gain 3 life </sentence> </line> </card>',
       "<card> <line> <sentence> flying this creature can't be blocked except by creatures with flying or reach </sentence> </line> <line> <sentence> when <CARD_NAME> enters the battlefield, destroy target enchantment </sentence> </line> </card>",
       '<card> <line> <precolon> <color> W,X </color> <color> T </color> </precolon> <sentence>  <CARD_NAME> deals x damage to target attacking or blocking creature </sentence> </line> </card>',
       '<card> <line> <sentence> prevent the next 1 damage that would be dealt to any target this turn </sentence> </line> <line> <sentence>

In [103]:
names = list(cards.keys())

In [77]:
colors = ["".join(cards[key]["colors"]) for key in list(cards.keys())]
colors = [color for color in colors if color != ""]
color_outputs = [[(1 if color_char in color else 0)/len(color) for color_char in ['B', 'G', 'R', 'U', 'W']] for color in colors]

In [78]:
colorful_names_and_texts = zip(names, lemmatized_card_texts)
colorful_names_and_texts = [(name, text) for name, text in colorful_names_and_texts if "".join(cards[name]["colors"]) != ""]
colorful_names, colorful_texts = [tup[0] for tup in colorful_names_and_texts], [tup[1] for tup in colorful_names_and_texts]

In [80]:
colorful_data = list(zip(colorful_names, [model[t] for t in colorful_texts], color_outputs))
colorful_test_data = random.sample(colorful_data, int(len(colorful_data)*0.3))
colorful_train_data = [d 
                       for d in colorful_data 
                       if d[0] not in set([x[0] for x in colorful_test_data])]

<ipython-input-80-d5acebe441ef>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  colorful_data = list(zip(colorful_names, [model[t] for t in colorful_texts], color_outputs))


KeyError: "word 'Ancestor's Chosen' not in vocabulary"

In [ ]:
RUNS_THROUGH = 5

class AllColorNet(torch.nn.Module):
    
    def __init__(self, layer_sizes, rho = 0.1):
        super(AllColorNet, self).__init__()
        self.rho = rho

        layer_sizes = [[layer_sizes[i], layer_sizes[i+1]] 
                       for i in range(len(layer_sizes)-1)]
        layers = []
        for i in range(len(layer_sizes)):
            layers.append(nn.Linear(*layer_sizes[i]))
            if i != len(layer_sizes)-1:
                layers.append(nn.ReLU())
        layers.append(nn.Softmax())
    
        self.linear_layers = torch.nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.linear_layers(x)
        x = torch.transpose(x, 1, 0)
        x = torch.mean(x, dim = 1)
        return x

    def fit(self, x, y,
          loss_function = torch.nn.functional.binary_cross_entropy_with_logits,
          rho = None):
        if rho == None:
            rho = self.rho
        opt = torch.optim.Adam(self.parameters(), lr = rho)
        for t in range(RUNS_THROUGH):
            pred = self(x)
            loss = loss_function(pred, y)
            opt.zero_grad()
            loss.backward()
            opt.step()

In [ ]:
net1 = AllColorNet([75, 150, 5], rho = 0.1)
for name, x, y in tqdm(colorful_train_data):
    x, y = torch.Tensor(x), torch.Tensor(y)
    net1.fit(x, y, loss_function = torch.nn.MSELoss())
    
    """print(name)
    text = cards[name]["text"]
    print(text)
    print(x.shape)
    print(y)
    print("\n")"""

In [ ]:
for name, x, y in colorful_test_data:
    print(name)
    text = cards[name]["text"]
    x, y = torch.Tensor(x), torch.Tensor(y)
    print(text)
    y_pred = net1(x)
    print(x.shape)
    print(y)
    print(y_pred)
    print("\n")


In [ ]:
RUNS_THROUGH = 5

class SingleColorNet(torch.nn.Module):
    
    def __init__(self, layer_sizes):
        super(SingleColorNet, self).__init__()

        layer_sizes = [[layer_sizes[i], layer_sizes[i+1]] 
                       for i in range(len(layer_sizes)-1)]
        layers = []
        for i in range(len(layer_sizes)):
            layers.append(nn.Linear(*layer_sizes[i]))
            if i != len(layer_sizes)-1:
                """layers.append(nn.ReLU())"""
        layers.append(nn.Tanh())
    
        self.linear_layers = torch.nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.linear_layers(x)
        return x

    def fit(self, x, y,
          loss_function = torch.nn.functional.binary_cross_entropy_with_logits,
          rho = None):
        if rho == None:
            rho = 0.1
        opt = torch.optim.Adam(self.parameters(), lr = rho)
        for t in range(RUNS_THROUGH):
            pred = self(x)
            loss = loss_function(pred, y)
            opt.zero_grad()
            loss.backward()
            opt.step()

In [ ]:
CURRENT_COLOR = "G" #BGRUW
net2 = SingleColorNet([75, 60, 30, 1])
for name, x, y in tqdm(colorful_train_data):
    x = torch.Tensor(x)
    x = torch.transpose(x, 1, 0)
    x = torch.mean(x, dim = 1)
    old_y = y.copy()
    y = torch.tensor(1.0 
                     if CURRENT_COLOR in cards[name]["text"]
                     else -1.0)
    net2.fit(x, y, loss_function = torch.nn.MSELoss(), rho = 0.1)

In [ ]:
d = {"-1,-1":0, "-1,1":0, "1,-1":0, "1,1":0}
for name, x, y in colorful_train_data:
    print(name)
    text = cards[name]["text"]
    x = torch.Tensor(x)
    x = torch.transpose(x, 1, 0)
    x = torch.mean(x, dim = 1)
    print(y)
    y = torch.tensor(1.0 
                     if CURRENT_COLOR in cards[name]["text"]
                     else -1.0)
    y_pred = net2(x)
    d[f"{int(y.item())},{int(y_pred.item())}"]+=1
    print(x.shape)
    print(y)
    print(y_pred)
    print("\n")
d

In [ ]:
for i in range(100):
    name = str(random.choice(list(cards.keys())))
    print(name)
    print(cards[name]["text"])
    print()

In [ ]:
N = 6
keys = set()
ngram_dict = {}
for card in tqdm(lemmatized_card_texts[0:int(len(lemmatized_card_texts))]):
    if len(card) < 5:
        continue
    for i in range(len(card)-(N-1)):
        ngram = " ".join(card[i:i+N])
        if ngram not in keys:
            keys.add(ngram)
            ngram_dict[ngram] = 0
        ngram_dict[ngram]+=1

ngram_dict = [[key, ngram_dict[key]] for key in list(ngram_dict.keys())]
ngram_dict.sort(key = lambda x: x[1], reverse = True)
ngram_dict = ngram_dict[0:10]
categories, counts = [c[0] for c in ngram_dict], [c[1] for c in ngram_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('N-gram Count')
ax.bar(categories, counts)
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
category_dict = {}
keys = set()
for name in list(cards.keys()):
    value = "".join(cards[name]["colors"])
    if value not in keys:
        keys.add(value)
        category_dict[value] = 0
    category_dict[value]+=1
del category_dict[""]

category_dict = {key: category_dict[key]/sum(list(category_dict.values())) for key in list(category_dict.keys()) if len(key) <= 2}
category_dict = [[key, category_dict[key]] for key in list(category_dict.keys())]
category_dict.sort(key = lambda x: x[1], reverse = True)
categories, counts = [c[0] for c in category_dict], [c[1] for c in category_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('Prominence of Color')
ax.bar(categories, counts)
plt.show()

In [ ]:
category_dict = {}
keys = set()
for name in list(cards.keys()):
    value = "".join(cards[name]["rarity"])
    if value not in keys:
        keys.add(value)
        category_dict[value] = 0
    category_dict[value]+=1

category_dict = {key: category_dict[key]/sum(list(category_dict.values())) for key in list(category_dict.keys())}
category_dict = [[key, category_dict[key]] for key in list(category_dict.keys())]
category_dict.sort(key = lambda x: x[1], reverse = True)
category_dict = category_dict[0:8]
categories, counts = [c[0] for c in category_dict], [c[1] for c in category_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('Prominence of Rarity')
ax.bar(categories, counts)
plt.show()

In [ ]:
category_dict = {}
keys = set()
for name in list(cards.keys()):
    value = "".join(cards[name]["type"])
    if value not in keys:
        keys.add(value)
        category_dict[value] = 0
    category_dict[value]+=1

category_dict = {key: category_dict[key]/sum(list(category_dict.values())) for key in list(category_dict.keys())}
category_dict = [[key, category_dict[key]] for key in list(category_dict.keys())]
category_dict.sort(key = lambda x: x[1], reverse = True)
category_dict = category_dict[0:8]
print(category_dict)
categories, counts = [c[0] for c in category_dict], [c[1] for c in category_dict]

fig = plt.figure()
ax = fig.add_axes([0,0,2,2])
ax.set_title('Prominence of Type')
ax.bar(categories, counts)
plt.show()

In [ ]:
for key in list(cards.keys()):
    print(key)

In [ ]:
cards["Denizen of the Deep"]["colors"]

In [ ]:
s = set()
d = {}
for key, color in [(key,str(cards[key]["rarity"])) for key in list(cards.keys())]:
    s.add(color)
    if color not in list(d.keys()):
        d[color] = 0
    d[color]+=1
print(d)

In [ ]:
for key in list(cards.keys()):
    if cards[key]["rarity"] == "rare":
        print(key)

In [ ]:
hints = set()
for key in list(cards.keys()):
    if "text" not in list(cards[key].keys()):
        continue
    text = cards[key]["text"]
    starts, ends = [], []
    for index, char in enumerate(list(text)):
        if char == "(":
            starts.append(index)
        elif char == ")":
            ends.append(index)
    for start, end in zip(starts, ends):
        hints.add(text[start+1:end])

In [ ]:
len(hints) / len(list(cards.keys()))

In [ ]:
cards["Chandra, Pyrogenius"]["manaCost"]
cards["Brown Ouphe"]

In [ ]:
print(len([x for x in model.wv.vocab]))
[x for x in model.wv.vocab]

In [ ]:
for index, (n,c,t) in enumerate(zip(names, colors, lemmatized_card_texts)):
    print(n,"\n", c, "\n", t, "\n\n")
    if index > 9:
        break

In [ ]:
[1,2,3,4,5,6]/6